## TFX - Deploy Training Pipeline to AI Platform Pipelines

You need to setup [AI Platform Pipelines](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) as a prerequisite, noting the **GKE Cluster Name**, **GKE Cluster Zone**, and the **GKE Namespace** of the Kubeflow pipelines installation.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import logging

logging.getLogger().setLevel(logging.INFO)

In [ ]:
PROJECT = 'ksalama-cloudml'
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us'

LOCAL_WORKSPACE = '_workspace'
if tf.io.gfile.exists(LOCAL_WORKSPACE):
    tf.io.gfile.rmtree(LOCAL_WORKSPACE)

## Set pipeline configurations

In [ ]:
os.environ["DATASET_DISPLAYNAME"] = 'chicago_taxi_tips'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/kfp_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

os.environ["GKE_CLUSTER_NAME"] = "ksalama-cloudml-dev-01"
os.environ["GKE_CLUSTER_ZONE"] = "europe-west2-a"
os.environ["NAMESPACE"] = "kubeflow-pipelines"

In [ ]:
from tfx_pipeline import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

## Build Container Image

In [ ]:
!echo $IMAGE_URI

In [ ]:
!gcloud builds submit --tag $IMAGE_URI . --timeout=15m

## Compile TFX Pipeline

In [ ]:
!rm $PIPELINE_NAME.tar.gz

In [ ]:
!tfx pipeline compile \
    --engine=kubeflow \
    --pipeline_path=tfx_pipeline/kfp_runner.py

## Deploy the Compiled Pipeline to KFP

In [ ]:
%%bash

gcloud container clusters get-credentials ${GKE_CLUSTER_NAME} --zone ${GKE_CLUSTER_ZONE}
export KFP_ENDPOINT=$(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

kfp --namespace=${NAMESPACE} --endpoint=${KFP_ENDPOINT} \
    pipeline upload \
    --pipeline-name=${PIPELINE_NAME} \
    ${PIPELINE_NAME}.tar.gz

In [ ]:
!echo $(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

![Pipeline execution](kfp-pipeline.png)